In [1]:
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

In [2]:
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

In [3]:
folder_path = '/content/drive/My Drive/'

##Import csv

In [4]:
filename = folder_path + 'O2_data.csv'
data = pd.read_csv(filename)

In [5]:
with pd.option_context('display.max_columns', None):
    display(data)

,unique_subject_id,unique_hospital_admission_id,unique_icustay_id,admission_age,sex_female,comorbidity_score_value,in_hospital_mortality,pH,pCO2,pO2,SaO2,SaO2_timestamp,SpO2,SpO2_timestamp,vitals_tempc,vitals_heart_rate,vitals_resp_rate,vitals_mbp_ni,vitals_mbp_i,cbc_hemoglobin,cbc_hematocrit,cbc_mch,cbc_mchc,cbc_mcv,cbc_platelet,cbc_rbc,cbc_rdw,cbc_wbc,coag_fibrinogen,coag_inr,coag_pt,coag_ptt,bmp_sodium,bmp_potassium,bmp_chloride,bmp_bicarbonate,bmp_bun,bmp_creatinine,bmp_glucose,bmp_aniongap,bmp_calcium,bmp_lactate,hfp_alt,hfp_alp,hfp_ast,hfp_bilirubin_total,hfp_albumin,sofa_past_overall_24hr,sofa_past_coagulation_24hr,sofa_past_liver_24hr,sofa_past_cardiovascular_24hr,sofa_past_cns_24hr,sofa_past_renal_24hr,sofa_past_respiratory_24hr,sofa_future_overall_24hr,sofa_future_respiratory_24hr,sofa_overall_increased,race_ethnicity_Asian,race_ethnicity_Black,race_ethnicity_HispanicLatino,race_ethnicity_Other,race_ethnicity_White
0,0,0,0,67.0,1,3.0,0.0,7.40,52.0,118.0,99.0,2014-01-01 02:54:00,99.0,2014-01-01 02:54:00,37.70,81.0,20.0,87.5,72.5,13.2,39.8,29.2,33.2,88.1,145.0,4.52,13.7,17.3,179.0,1.1,11.6,25.0,140.0,3.1,101.0,25.0,13.5,0.90,100.0,10.0,9.5,1.05,31.5,95.5,25.0,0.65,4.4,9.0,1.0,0.0,1.0,4.0,0.0,3.0,2.0,0,False,False,False,False,False,True
1,0,0,0,67.0,1,3.0,0.0,7.38,53.0,93.0,97.0,2014-01-01 03:39:00,98.0,2014-01-01 03:39:00,37.90,82.0,21.0,87.5,78.5,13.2,39.8,29.2,33.2,88.1,145.0,4.52,13.7,17.3,179.0,1.1,11.6,25.0,140.0,3.1,101.0,25.0,13.5,0.90,119.0,10.0,9.5,1.05,31.5,95.5,25.0,0.65,4.4,9.0,1.0,0.0,1.0,4.0,0.0,3.0,2.0,0,False,False,False,False,False,True
2,3,3,3,73.0,0,4.0,0.0,7.28,52.0,73.0,92.0,2014-01-03 23:47:00,96.0,2014-01-03 23:45:00,37.30,96.0,27.0,87.5,68.0,11.4,34.1,29.8,33.4,89.0,100.0,3.83,12.6,9.5,300.0,1.1,10.8,39.0,145.0,3.6,113.0,21.0,11.0,1.11,123.0,15.0,7.3,1.05,31.5,95.5,25.0,1.10,2.5,4.0,1.0,0.0,1.0,0.0,0.0,2.0,4.0,0,False,True,False,False,False,False
3,3,3,3,73.0,0,4.0,0.0,7.36,40.0,157.0,99.0,2014-01-03 20:53:00,100.0,2014-01-03 20:50:00,36.00,83.0,11.0,87.5,68.0,11.4,34.1,29.8,33.4,89.0,100.0,3.83,12.6,9.5,300.0,1.1,10.8,39.0,145.0,3.6,113.0,21.0,11.0,1.11,119.0,15.0,7.3,1.05,31.5,95.5,25.0,1.10,2.5,3.0,1.0,0.0,0.0,0.0,0.0,2.0,4.0,0,False,True,False,False,False,False
4,3,3,3,73.0,0,4.0,0.0,7.32,45.0,74.0,93.0,2014-01-04 00:41:00,97.0,2014-01-04 00:40:00,37.80,99.0,18.0,87.5,69.0,11.4,34.1,29.8,33.4,89.0,100.0,3.83,12.6,9.5,300.0,1.1,10.8,39.0,145.0,3.6,113.0,21.0,11.0,1.11,123.0,15.0,7.3,1.05,31.5,95.5,25.0,1.10,2.5,4.0,1.0,0.0,1.0,0.0,0.0,2.0,4.0,0,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163391,44897,49088,51959,68.0,0,3.0,0.0,7.39,49.0,87.0,93.0,2126-10-23 19:00:00+00:00,95.0,2126-10-23 19:00:00+00:00,37.89,80.0,24.0,87.5,67.0,8.7,28.4,26.2,30.6,86.0,209.0,3.32,20.7,11.0,277.0,1.3,14.6,37.3,139.0,4.6,99.0,28.0,29.0,1.70,126.0,12.0,7.6,1.05,31.5,76.0,38.0,0.80,2.3,6.0,0.0,0.0,1.0,0.0,1.0,4.0,7.0,1,False,False,False,False,True,False
163392,44898,49089,51960,84.0,0,5.0,0.0,7.46,33.0,64.0,93.0,2125-10-21 11:03:00+00:00,96.0,2125-10-21 11:00:00+00:00,37.39,117.0,22.0,76.0,68.0,11.2,33.9,28.3,33.1,86.0,191.0,3.96,13.7,14.9,223.0,1.4,14.8,40.0,134.0,5.0,106.0,24.0,11.0,0.80,145.0,9.0,7.3,1.60,31.5,95.5,25.0,0.65,4.4,5.0,0.0,0.0,1.0,2.0,0.0,2.0,5.0,0,False,False,False,False,False,True
163393,44899,49090,51961,76.0,0,10.0,0.0,7.38,51.0,96.0,97.0,2123-12-30 10:03:00+00:00,98.0,2123-12-30 10:00:00+00:00,36.89,83.0,16.0,87.5,87.0,9.6,30.6,29.9,31.4,95.0,355.0,3.21,15.9,15.2,300.0,1.4,14.6,140.4,136.0,4.0,98.0,30.0,46.0,1.70,196.0,12.0,7.9,1.10,22.0,187.0,39.0,0.40,4.4,11.0,0.0,0.0,3.0,4.0,2.0,2.0,5.0,1,False,False,False,False,False,True
163394,44900,49091,51962,63.0,1,7.0,0.0,7.44,33.0,110.0,96.0,2126-05-06 08:03:00+00:00,99.0,2126-05-06 08:00:00+00:00,36.89,89.0,1

#Feature selection

In [6]:
def getSelectedFeatures(method = []):
    selected_columns = []

    if method == 'mySelection':
        selected_columns = ['unique_hospital_admission_id',
                            'admission_age', 'sex_female',
                            'comorbidity_score_value',
                            'SaO2',	'SpO2',
                            'vitals_tempc',
                            'vitals_heart_rate',
                            'vitals_resp_rate',
                            'vitals_mbp_ni',
                            'vitals_mbp_i',
                            'cbc_hemoglobin',
                            'cbc_platelet',
                            'cbc_wbc',
                            'bmp_sodium',
                            'bmp_potassium',
                            'bmp_bun',
                            'bmp_creatinine',
                            'bmp_bicarbonate',
                            'bmp_glucose',
                            'bmp_aniongap',
                            'bmp_lactate',
                            'hfp_albumin',
                            'sofa_past_overall_24hr',
                            'sofa_past_respiratory_24hr',
                            'sofa_past_cardiovascular_24hr',
                            'race_ethnicity_Asian',
                            'race_ethnicity_Black',
                            'race_ethnicity_HispanicLatino',
                            'race_ethnicity_Other',
                            'race_ethnicity_White'
                            ]

    return selected_columns

#Models

In [7]:
# Imports
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedGroupKFold

from sklearn.metrics import roc_auc_score, f1_score, recall_score, accuracy_score
import scipy.stats as st

from sklearn.utils.class_weight import compute_sample_weight

In [8]:
models = [
        LogisticRegression()
        ,
        xgb.XGBClassifier(objective= "binary:logistic")
          ]

##metricsByRace

In [11]:
def metricsByRace(x_test, y_test, prob, prob_bin):
    race_ethnicity_columns = ['race_ethnicity_Asian',
                            'race_ethnicity_Black',
                            'race_ethnicity_HispanicLatino',
                            'race_ethnicity_Other',
                            'race_ethnicity_White']
    #AUC ROC
    roc_aucByRace = []
    #Recall
    recall_byRace = []
    #F1
    f1Scores_byRace = []
    #Accuracy
    accuracy_byRace = []

    for race_ethn in race_ethnicity_columns:
        try:
            rocAuc = roc_auc_score(y_test[x_test[race_ethn] == 1], prob[x_test[race_ethn] == 1])
        except:
            rocAuc = np.nan
        roc_aucByRace.append(rocAuc)
        recall_byRace.append(recall_score(y_test[x_test[race_ethn] == 1], prob_bin[x_test[race_ethn] == 1], average = 'binary'))
        f1Scores_byRace.append(f1_score(y_test[x_test[race_ethn] == 1], prob_bin[x_test[race_ethn] == 1], average = 'binary'))
        accuracy_byRace.append(accuracy_score(y_test[x_test[race_ethn] == 1], prob_bin[x_test[race_ethn] == 1]))

    return [round(k, 3) for k in roc_aucByRace], [round(y, 3) for y in recall_byRace], [round(x, 3) for x in f1Scores_byRace], [round(z, 3) for z in accuracy_byRace]

##metricsByBiasBin

In [12]:
def metricsByBiasBin(x_test, y_test, prob, prob_bin, bias_bins):
    #AUC ROC
    roc_auc = []
    #Recall
    recall = []
    #F1
    f1Score = []
    #Accuracy
    accuracy = []

    for min, max in bias_bins:
        try:
            rocAuc = roc_auc_score(y_test[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)], prob[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)])
        except:
            rocAuc = np.nan
        roc_auc.append(rocAuc)
        recall.append(recall_score(y_test[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)], prob_bin[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)], average = 'binary'))
        f1Score.append(f1_score(y_test[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)], prob_bin[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)], average = 'binary'))
        accuracy.append(accuracy_score(y_test[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)], prob_bin[(x_test['SpO2-SaO2'] >= min) & (x_test['SpO2-SaO2'] < max)]))

    return [round(k, 3) for k in roc_auc], [round(y, 3) for y in recall], [round(x, 3) for x in f1Score], [round(z, 3) for z in accuracy]

##metricsByHH

In [13]:
def metricsByHH(x_test, y_test, prob, prob_bin):
    #AUC ROC
    roc_auc = []
    #Recall
    recall = []
    #F1
    f1Score = []
    #Accuracy
    accuracy = []

    for i in [0,1]:
        if i == 0:
            index = ((x_test['SpO2'] >= 88) & (x_test['SaO2'] >= 88)) #consistent values
        else:
            index = ((x_test['SpO2'] >= 88) & (x_test['SaO2'] < 88)) #hidden hypoxemia

        try:
            roc_auc.append(roc_auc_score(y_test[index], prob[index]))
        except:
            roc_auc.append(np.nan)
        recall.append(recall_score(y_test[index], prob_bin[index], average = 'binary'))
        f1Score.append(f1_score(y_test[index], prob_bin[index], average = 'binary'))
        accuracy.append(accuracy_score(y_test[index], prob_bin[index]))

    return [round(k, 3) for k in roc_auc], [round(y, 3) for y in recall], [round(x, 3) for x in f1Score], [round(z, 3) for z in accuracy]

###getMetrics

In [14]:
def getMetrics(y, prob, prob_bin):
    try:
        roc_auc = roc_auc_score(y, prob)
    except:
        roc_auc = np.nan
    recall = recall_score(y, prob_bin, average = 'binary')
    f1Score = f1_score(y, prob_bin, average = 'binary')
    accuracy = accuracy_score(y, prob_bin)
    return [round(x, 3) for x in [roc_auc, recall, f1Score, accuracy]]

##calculateMetrics

In [18]:
def calculateMetrics(model, x_data, X, Y, reference_feature, biased_feature, min_max):
    X_ = X[:]
    prob_bin = model.predict(x_data)
    prob = model.predict_proba(x_data)[:, 1]

    metrics = getMetrics(Y, prob, prob_bin)

    metrics_ByRace = metricsByRace(X_, Y, prob, prob_bin)

    bias_column = 'SpO2-SaO2'
    bias_bins = {}
    X_[bias_column] = X_[biased_feature] - X_[reference_feature]
    for min, max in min_max:
        bias_bins[str(min) + '_' + str(max)] = (X_[bias_column] >= min) & (X_[bias_column] < max)

    metrics_ByBiasBin = metricsByBiasBin(X_, Y, prob, prob_bin, min_max)

    metrics_ByHH = metricsByHH(X_, Y, prob, prob_bin)

    return np.hstack(([metrics[0]] + metrics_ByRace[0] + \
            [metrics[1]] + metrics_ByRace[1] + \
            [metrics[2]] + metrics_ByRace[2] + \
            [metrics[3]] + metrics_ByRace[3], \
            np.reshape(metrics_ByBiasBin, (1,-1))[0], \
            np.reshape(metrics_ByHH, (1,-1))[0]))

###calculateStatistics

In [17]:
def calculateStatistics(data):
    results = round(np.nanmean(data), 3)
    return results

##getPValue

In [10]:
# Paired-sample t-test
def getPValue(group1, group2):
    _, p_value = st.ttest_rel(group1, group2, alternative = 'two-sided', nan_policy='omit')
    CI = st.ttest_rel(group1, group2, alternative = 'two-sided', nan_policy='omit').confidence_interval(confidence_level=0.95)

    if p_value <.001:
        pvalue = '<.001'
    elif p_value < .01:
        pvalue = '<.01'
    else:
        pvalue = round(p_value, 3)
    return pvalue, round(CI[0], 3), round(CI[1], 3)

#save_results

In [20]:
def save_results(path, model, results, O2_groups, tasks, reference_feature):
    n_metrics = 4*6 + 2 + 4*4 + 4*2
    new_results = []

    for task in tasks:
        for ft in O2_groups:
            for t in ['train', 'test']:
                new_results.append(results[0][t][task][ft])
                if (ft != reference_feature) & (t == 'test'):
                    pvalue_test = []
                    CI_test = []
                    for metric in range(n_metrics):
                        pvalue_test.append((results[1][task][ft][metric])[0])
                        CI_test.append(f"({(results[1][task][ft][metric])[1]} - {(results[1][task][ft][metric])[2]})")
                    new_results.append(pvalue_test)
                    new_results.append(CI_test)

    df_existing = pd.read_excel(path)
    df_combined = pd.concat([df_existing, pd.DataFrame([[model,
                                                            datetime.now().strftime("%d/%m/%Y %H:%M:%S")
                                                            ]]),
                                pd.DataFrame(new_results)],
                            ignore_index=True) # Append new data
    df_combined.to_excel(path, index=False) # Save the combined data to Excel

#Plots

In [23]:
def defineMetricsAndGroups(bias_bins):
    metricNames = ['AUROC', 'Recall', 'F1-score', 'Accuracy']
    tasksNames = ['Mortality', 'Respiratory\nSOFA score', 'Overall SOFA\nscore increase']
    metrics = np.arange(len(metricNames))
    race_groups = ['All','A','B','HL','O','W']

    bias_groups = []
    for min, max in bias_bins:
        if ([min, max] == bias_bins[0]):
            bias_groups.append('(-$\infty$, ' + str(max) + ')')
        elif ([min, max] == bias_bins[3]):
            bias_groups.append('[' + str(min) + ', +$\infty$)')
        else:
            bias_groups.append('[' + str(min) + ', ' + str(max) + ')')

    group_hidden_hypoxemia = [0, 1]

    return tasksNames, metricNames, metrics, race_groups, bias_groups, group_hidden_hypoxemia

###plotBarChart

In [24]:
def plotSignificantValues(ax, x, y, p_values):
    for k in x:
        if p_values[k] == '<.001':
            ax.text(x[k], y+0.1, '***', fontsize=15, weight='bold', ha='center')
        elif p_values[k] == '<.01':
            ax.text(x[k], y+0.1, '**', fontsize=15, weight='bold', ha='center')
        elif float(p_values[k]) < 0.05:
            ax.text(x[k], y+0.1, '*', fontsize=15, weight='bold', ha='center')

In [25]:
def plotBarChart(figs, axs, data, model, task, tasks, reference_feature, biased_feature, bias_bins):
    tasksNames, metricNames, metrics, race_groups, bias_groups, group_hidden_hypoxemia = defineMetricsAndGroups(bias_bins)
    groups = [race_groups, bias_groups, group_hidden_hypoxemia]
    width = 0.4

    for group in [0,1,2]:
        for metric in metrics:
            ax = axs[group][metric, tasks.index(task)]
            x = np.arange(len(groups[group]))
            if group == 0:
                y1 = np.array(data[0][reference_feature])[(metric)*len(race_groups) : (metric)*len(race_groups)+len(race_groups)]
                y2 = np.array(data[0][biased_feature])[(metric)*len(race_groups) : (metric)*len(race_groups)+len(race_groups)]
                p_values = np.array(data[1][biased_feature])[(metric)*len(race_groups) : (metric)*len(race_groups)+len(race_groups) , 0]
            elif group == 1:
                y1 = np.array(data[0][reference_feature])[(metric)*len(bias_groups) + len(metricNames)*len(race_groups) +2 : (metric)*len(bias_groups)+len(bias_groups) + len(metricNames)*len(race_groups) +2]
                y2 = np.array(data[0][biased_feature])[(metric)*len(bias_groups)  + len(metricNames)*len(race_groups) +2 : (metric)*len(bias_groups)+len(bias_groups) + len(metricNames)*len(race_groups) +2]
                p_values = np.array(data[1][biased_feature])[(metric)*len(bias_groups) + len(metricNames)*len(race_groups) +2 : (metric)*len(bias_groups)+len(bias_groups) + len(metricNames)*len(race_groups) +2 , 0]
            else:
                y1 = np.array(data[0][reference_feature])[(metric)*len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+ len(bias_groups)) +2 : (metric)*len(group_hidden_hypoxemia)+len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+len(bias_groups)) +2]
                y2 = np.array(data[0][biased_feature])[(metric)*len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+ len(bias_groups)) +2 : (metric)*len(group_hidden_hypoxemia)+len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+len(bias_groups)) +2]
                p_values = np.array(data[1][biased_feature])[(metric)*len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+ len(bias_groups)) +2 : (metric)*len(group_hidden_hypoxemia)+len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+len(bias_groups)) +2, 0]

            ax.bar(x-width/2, y1, width, color='tab:red', alpha=0.9)
            ax.bar(x+width/2, y2, width, color='tab:blue', alpha= 0.9, edgecolor='w', hatch ='/')

            if group == 1:
                ax.axvline(x=1.5, linestyle='--', color='grey', linewidth = 0.5)

            for x_value in x:
               ax.text(x_value-width/2, y1[x_value], "%.2f" % y1[x_value], fontsize=8, ha='center', va='bottom')
               ax.text(x_value+width/2, y2[x_value], "%.2f" % y2[x_value], fontsize=8, ha='center', va='bottom')

            # Plot asterisks
            y_asterisk = np.array([y1, y2]).max()
            plotSignificantValues(ax, x, y_asterisk, p_values)

            ax.grid(True, axis='y', linestyle='--', color='grey', linewidth = 0.5)

            if tasks.index(task) == 0:
                ax.set_ylabel(metricNames[metric])

            if metric == 0:
                ax.set_title(tasksNames[tasks.index(task)])

        ax.set_xticks(x, labels=groups[group])

        if tasks.index(task) == (len(tasks)-1):
            if group == 1:
                figs[group].legend(['_legend', 'Model with SaO$_{2}$', 'Model with SpO$_{2}$'],
                                loc='lower center',
                                bbox_to_anchor=(0.54, -0.05),
                                fancybox=False, ncol=2)
            else:
                figs[group].legend(['Model with SaO$_{2}$', 'Model with SpO$_{2}$'],
                                loc='lower center',
                                bbox_to_anchor=(0.54, -0.05),
                                fancybox=False, ncol=2)
            ax.set_ylim([0, 1])
            if group == 0:
                figs[group].supxlabel('Race and ethnicity subgroups', x=0.54)
            elif group == 1:
                for i in range(len(tasks)):
                    axs[group][metric, i].text((ax.get_xlim()[1] + ax.get_xlim()[0])*0.47, ax.get_ylim()[0]-0.15, r'$\leftarrow$''SpO$_{2}$\nunderestimates',
                            ha='right', va='top', fontsize=8)
                    axs[group][metric, i].text((ax.get_xlim()[1] + ax.get_xlim()[0])*0.53, ax.get_ylim()[0]-0.15, 'SpO$_{2}$'r'$\rightarrow$''\noverestimates',
                            ha='left', va='top', fontsize=8)
                figs[group].supxlabel('SpO$_{2}$ - SaO$_{2}$ bins', x=0.54)
            else:
                figs[group].supxlabel('Hidden Hypoxemia', x=0.54)
            figs[group].supylabel('Performance metrics (mean)')
            figs[group].savefig(folder_path + 'Plots/' + model + str(group) + biased_feature + '_BarChart_new.png', bbox_inches='tight')
            figs[group].show()

##plotDeltaMetric

In [26]:
def plotDeltaMetric(figs, axs, data, model, task, tasks, biased_feature, bias_bins):
    tasksNames, metricNames, metrics, race_groups, bias_groups, group_hidden_hypoxemia = defineMetricsAndGroups(bias_bins)
    groups = [race_groups, bias_groups, group_hidden_hypoxemia]

    y_max = [0.1, 0.3, 0.15, 0.1]

    for group in [0,1,2]:
        for metric in metrics:
            ax = axs[group][metric, tasks.index(task)]
            x = np.arange(len(groups[group])) +1
            if group == 0:
                y = np.array(data[biased_feature])[:, (metric)*len(race_groups) : (metric)*len(race_groups)+len(race_groups)]
            elif group == 1:
                y = np.array(data[biased_feature])[:, (metric)*len(bias_groups) + len(metricNames)*len(race_groups) +2 : (metric)*len(bias_groups)+len(bias_groups) + len(metricNames)*len(race_groups) +2]
            else:
                y = np.array(data[biased_feature])[:, (metric)*len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+ len(bias_groups)) +2 : (metric)*len(group_hidden_hypoxemia)+len(group_hidden_hypoxemia) + len(metricNames)*(len(race_groups)+len(bias_groups)) +2]

            ax.boxplot(y, medianprops=dict(color='k', linewidth = 1.2), patch_artist=True,
                            boxprops=dict(facecolor='tab:blue', alpha=0.9, color='k'),
                            capprops=dict(color='k', linewidth = 0.5),
                            whiskerprops=dict(color='k', linewidth = 0.5))
            ax.axhline(y = 0, color = 'k', linestyle = 'dashed', linewidth = 0.5)
            ax.grid(True, axis='y', linestyle='--', color='grey', linewidth = 0.5)

            if group == 1:
                ax.axvline(x=1.5, linestyle='--', color='grey', linewidth = 0.5)

            if tasks.index(task) == 0:
                ax.set_ylabel(metricNames[metric])
                ax.set_ylim(ymin=-y_max[metric], ymax=y_max[metric])

            if metric == 0:
                ax.set_title(tasksNames[tasks.index(task)])

            if tasks.index(task) == (len(tasks)-1): #last task
                ax.text(len(groups[group])+0.6, 0+0.05, r'$\uparrow$''SpO$_{2}$ model\nperforms better',
                        ha='left', va='bottom', fontsize=8)
                ax.text(len(groups[group])+0.6, 0-0.05, r'$\downarrow$''SaO$_{2}$ model\nperforms better',
                        ha='left', va='top', fontsize=8)

        ax.set_xticks(x, labels=groups[group])

        if tasks.index(task) == (len(tasks)-1): #last task
            if group == 0:
                figs[group].supxlabel('Race and ethnicity subgroups')
            elif group == 1:
                for i in range(len(tasks)):
                    axs[group][metric, i].text((ax.get_xlim()[1] + ax.get_xlim()[0])*0.47, ax.get_ylim()[0]*1.3, r'$\leftarrow$''SpO$_{2}$\nunderestimates',
                            ha='right', va='top', fontsize=8)
                    axs[group][metric, i].text((ax.get_xlim()[1] + ax.get_xlim()[0])*0.53, ax.get_ylim()[0]*1.3, 'SpO$_{2}$'r'$\rightarrow$''\noverestimates',
                            ha='left', va='top', fontsize=8)
                figs[group].supxlabel('SpO$_{2}$ - SaO$_{2}$ bins', x=0.54)
            else:
                figs[group].supxlabel('Hidden Hypoxemia')

            figs[group].supylabel('Difference between SpO$_{2}$ and SaO$_{2}$ models\' performance')
            figs[group].savefig(folder_path + 'Plots/' + model + str(group) + biased_feature + '_Delta_new.png', bbox_inches='tight')
            figs[group].show()

##getModelResults

In [19]:
def getModelResults(x, y, k_fold, modelName, tasks, reference_feature, biased_feature, bias_bins):
    groups = np.array(x['unique_subject_id'])
    skf = StratifiedGroupKFold(n_splits=k_fold)

    race_ethnicity_columns = ['race_ethnicity_Asian',
                            'race_ethnicity_Black',
                            'race_ethnicity_HispanicLatino',
                            'race_ethnicity_Other',
                            'race_ethnicity_White']
    O2_column = [reference_feature, biased_feature]
    all_metrics = {}
    delta_test = {}
    metrics_Statistics = {}
    pvalue = {}

    # Evaluation Metrics and Variables to record data
    for t in ['train', 'test']:
        all_metrics[t] = {}
        metrics_Statistics[t] = {}
        for task in tasks:
            all_metrics[t][task] = {}
            delta_test[task] = {}
            metrics_Statistics[t][task] = {}
            pvalue[task] = {}
            for ft in O2_column:
                all_metrics[t][task][ft] = []
                delta_test[task][ft] = []
                metrics_Statistics[t][task][ft] = []
                pvalue[task][ft] = []

    feat_sel_model = 'mySelection'
    selected_features = getSelectedFeatures(feat_sel_model)
    x = x[selected_features]

    for task in tasks:
        print('task: ', task)
        y_task = y[task]

        for fold, (train_index, test_index) in enumerate(skf.split(x, y_task, groups)):
            print('fold: ', fold)
            X_train, Y_train = x.iloc[train_index], y_task.iloc[train_index]
            X_test, Y_test = x.iloc[test_index], y_task.iloc[test_index]

            for feature in O2_column:
                print('Feature: ', feature)
                if feature != reference_feature:
                    x_data_train = X_train.drop(race_ethnicity_columns + [reference_feature, 'unique_hospital_admission_id'], axis=1)
                    x_data_test = X_test.drop(race_ethnicity_columns + [reference_feature, 'unique_hospital_admission_id'], axis=1)
                else:
                    x_data_train = X_train.drop(race_ethnicity_columns + [biased_feature, 'unique_hospital_admission_id'], axis=1)
                    x_data_test = X_test.drop(race_ethnicity_columns + [biased_feature, 'unique_hospital_admission_id'], axis=1)

                # for task in tasks:
                modelName.fit(x_data_train, Y_train, sample_weight = compute_sample_weight(class_weight="balanced", y=Y_train))
                #Train
                all_metrics['train'][task][feature].append(calculateMetrics(modelName, x_data_train, X_train, Y_train, reference_feature, biased_feature, bias_bins))
                #Test
                all_metrics['test'][task][feature].append(calculateMetrics(modelName, x_data_test, X_test, Y_test, reference_feature, biased_feature, bias_bins))

    for task in tasks:
        for ft in O2_column:
            if (ft != reference_feature):
            # Calculate difference between metrics
                delta_test[task][ft] = np.reshape(all_metrics['test'][task][ft], (k_fold,-1)) - np.reshape(all_metrics['test'][task][reference_feature], (k_fold,-1))

    total_metrics = 4*6 + 4*4 + 4*2
    for t in ['train', 'test']:
        for task in tasks:
            for ft in O2_column:
                for metric in range(total_metrics):
                    metrics_Statistics[t][task][ft].append(calculateStatistics(np.reshape(all_metrics[t][task][ft], (k_fold,-1))[:,metric]))
                    if (ft != reference_feature) & (t == 'test'):
                        # Compute p-value
                        pvalue[task][ft].append(getPValue(np.reshape(all_metrics[t][task][reference_feature], (k_fold,-1))[:,metric],
                                                            np.reshape(all_metrics[t][task][ft], (k_fold,-1))[:,metric]))

    # save model's results
    save_results(folder_path + 'ModelResults.xlsx', modelName.__class__.__name__, [metrics_Statistics, pvalue], O2_column, tasks, reference_feature)

    for ft in O2_column:
        if ft != reference_feature:
            fig1, ax1 = plt.subplots(4, len(tasks), figsize=(15,10), sharex=True, sharey=True, layout="constrained")
            fig2, ax2 = plt.subplots(4, len(tasks), figsize=(15,10), sharex=True, sharey=True, layout="constrained")
            fig2_1, ax2_1 = plt.subplots(4, len(tasks), figsize=(10,10), sharex=True, sharey=True, layout="constrained")

            fig3, ax3 = plt.subplots(4, len(tasks), figsize=(15,10), sharex=True, sharey='row', layout="constrained")
            fig4, ax4 = plt.subplots(4, len(tasks), figsize=(15,10), sharex=True, sharey='row', layout="constrained")
            fig4_1, ax4_1 = plt.subplots(4, len(tasks), figsize=(15,10), sharex=True, sharey='row', layout="constrained")

            for task in tasks:
                # Plots
                plotBarChart([fig1, fig2, fig2_1], [ax1, ax2, ax2_1], [metrics_Statistics['test'][task], pvalue[task]], str(modelName.__class__.__name__), task, tasks, reference_feature, ft, bias_bins=bias_bins)
                plotDeltaMetric([fig3, fig4, fig4_1], [ax3, ax4, ax4_1], delta_test[task], str(modelName.__class__.__name__), task, tasks, ft, bias_bins=bias_bins)

#Data split

In [27]:
def xy_split(data, tasks):
    x_data = data.drop(tasks, axis= 1)
    y_data = data[tasks]
    return x_data, y_data

#Main

In [28]:
tasks = ['in_hospital_mortality', 'sofa_future_respiratory_24hr', 'sofa_overall_increased']

In [29]:
reference_feature = 'SaO2'
biased_feature = 'SpO2'

In [30]:
all_x_data, all_y_task = xy_split(data.drop(['unique_icustay_id', 'sofa_future_overall_24hr','SaO2_timestamp', 'SpO2_timestamp'], axis= 1), tasks)

##Train + Test

In [ ]:
kfold = 10
min_max = [[float('-inf'), -3],
               [-3, 0],
               [0, 3],
               [3, float('inf')]]

for model in models:
    display(str(model.__class__.__name__))
    metrics_ = getModelResults(all_x_data, all_y_task, kfold,
                        model, tasks, reference_feature, biased_feature, min_max)